In [1]:
import pandas as pd

In [2]:
ICD10_MAP = {
    'C50': 'Breast',
    'C51': 'Vulva',
    'C52': 'Vagina',
    'C53': 'Cervix Uteri',
    'C54': 'Corpus Uteri',
    'C55': 'Uterus',
    'C56': 'Ovary',
    'C57': 'Other and Unspecified Female Genital Organs',
}

In [3]:
df = pd.read_csv('heisin_dag.csv')

In [20]:
LEVEL = 'level'
DIAG = 'diag_icd10'
FEMAL_CANCER_TYPE = 'cancer_type'
df[FEMAL_CANCER_TYPE] =df.apply(lambda x: ICD10_MAP.get(str(x[DIAG])[:3], None), axis=1)

primary_fc = df[(df[LEVEL] == 'Primary/main diagnosis') & (df[DIAG].str.contains('C5', na=False))]

In [21]:
dates = pd.read_csv('bc_hesin.csv', parse_dates=['epistart'])[['dnx_hesin_id', 'epistart']] 
primary_fc = primary_fc.merge(dates, on='dnx_hesin_id', how='left')
primary_fc.info(show_counts=True)

/var/folders/11/krx912jd2nqgvltq6nl2ghc00000gn/T/ipykernel_2932/3330282795.py:1: DtypeWarning: Columns (3,19,36) have mixed types. Specify dtype option on import or set low_memory=False.
  dates = pd.read_csv('bc_hesin.csv', parse_dates=['epistart'])[['dnx_hesin_id', 'epistart']]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142992 entries, 0 to 142991
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   dnx_hesin_diag_id  142992 non-null  object        
 1   dnx_hesin_id       142992 non-null  object        
 2   eid                142992 non-null  int64         
 3   ins_index          142992 non-null  int64         
 4   arr_index          142992 non-null  int64         
 5   level              142992 non-null  object        
 6   diag_icd9          0 non-null       object        
 7   diag_icd9_nb       0 non-null       float64       
 8   diag_icd10         142992 non-null  object        
 9   diag_icd10_nb      32 non-null      object        
 10  cancer_type        142992 non-null  object        
 11  epistart           142831 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 13.1+ MB


In [22]:
pivot_df = primary_fc.pivot_table(index='eid', 
                          columns=FEMAL_CANCER_TYPE, 
                          values='epistart', 
                          aggfunc='min', 
                          fill_value=None).reset_index()
pivot_df['number_cancer'] = pivot_df[ICD10_MAP.values()].count(axis=1)
# Display result
pivot_df.groupby('number_cancer').nunique('eid')

cancer_type,eid,Breast,Cervix Uteri,Corpus Uteri,Other and Unspecified Female Genital Organs,Ovary,Uterus,Vagina,Vulva
number_cancer,,,,,,,,,
1,19401,6280,2,29,20,1290,2,0,3
2,638,414,27,302,146,314,18,6,26
3,49,38,8,37,17,25,18,2,2
4,5,5,2,5,2,2,4,0,0


In [24]:
# does everyone here have breast cancer? 
display(len(set(primary_fc['eid'])))
pivot_df['has_ovarian'] = pivot_df['Ovary'].notna()
pivot_df['has_breast'] = pivot_df['Breast'].notna()
display(pivot_df.groupby(['has_breast','has_ovarian'])['eid'].nunique('eid'))
display(pivot_df[(pivot_df['has_breast'] == False) & (pivot_df['has_ovarian'] == False) & (pivot_df['number_cancer'] > 0)])


20125

has_breast  has_ovarian
False       False             59
            True            1686
True        False          18223
            True             125
Name: eid, dtype: int64

cancer_type,eid,Breast,Cervix Uteri,Corpus Uteri,Other and Unspecified Female Genital Organs,Ovary,Uterus,Vagina,Vulva,number_cancer,has_ovarian,has_breast
921,1227790,NaT,NaT,NaT,2014-10-09,NaT,NaT,NaT,NaT,1,False,False
986,1243749,NaT,NaT,2020-01-07,NaT,NaT,NaT,NaT,NaT,1,False,False
1030,1254962,NaT,NaT,1999-10-14,NaT,NaT,NaT,NaT,NaT,1,False,False
1328,1328618,NaT,NaT,NaT,NaT,NaT,2021-01-01,NaT,NaT,1,False,False
1684,1416997,NaT,NaT,NaT,2013-06-05,NaT,NaT,NaT,NaT,1,False,False
1805,1450537,NaT,NaT,2008-09-09,NaT,NaT,NaT,NaT,NaT,1,False,False
2300,1565971,NaT,NaT,2005-02-03,NaT,NaT,NaT,NaT,NaT,1,False,False
3249,1802883,NaT,NaT,NaT,2021-02-15,NaT,NaT,NaT,NaT,1,False,False
3281,1810053,NaT,NaT,2011-10-04,NaT,NaT,NaT,NaT,NaT,1,False,False
4052,2003701,NaT,NaT,NaT,2021-02-26,NaT,NaT,NaT,NaT,1,False,False


In [28]:
display(df[(df[DIAG].str.contains('C5', na=False)) & (df['eid'] == 2559548)])

,dnx_hesin_diag_id,dnx_hesin_id,eid,ins_index,arr_index,level,diag_icd9,diag_icd9_nb,diag_icd10,diag_icd10_nb,cancer_type
99396,2559548-16-0,2559548-16,2559548,16,0,Primary/main diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99408,2559548-17-5,2559548-17,2559548,17,5,Secondary diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99419,2559548-19-7,2559548-19,2559548,19,7,Secondary diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99430,2559548-22-4,2559548-22,2559548,22,4,Secondary diagnosis,NaN,NaN,C56 Malignant neoplasm of ovary,NaN,Ovary
99434,2559548-25-0,2559548-25,2559548,25,0,Primary/main diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99443,2559548-26-0,2559548-26,2559548,26,0,Primary/main diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99462,2559548-9-0,2559548-9,2559548,9,0,Primary/main diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99483,2559548-14-0,2559548-14,2559548,14,0,Primary/main diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99507,2559548-13-4,2559548-13,2559548,13,4,Secondary diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs
99516,2559548-28-0,2559548-28,2559548,28,0,Primary/main diagnosis,NaN,NaN,C57.8 Overlapping lesion of female genital organs,NaN,Other and Unspecified Female Genital Organs


In [32]:
sum_of_true_values = pivot_df.drop(columns='eid').count()
print("Sum of True values per cancer type:")
print(sum_of_true_values)

Sum of True values per cancer type:
cancer_type
Breast                                         18348
Cervix Uteri                                      39
Corpus Uteri                                     377
Other and Unspecified Female Genital Organs      188
Ovary                                           1811
Uterus                                            42
Vagina                                             8
Vulva                                             31
number_cancer                                  20093
dtype: int64


In [33]:
pivot_df.to_csv('cleaned_diagnosis_data.csv')